<h1 align="center">中文資訊檢索（文件檢索）採用 TF-IDF 方法</h1>
<h3 align="center">Chinese Document Retrieval with TF-IDF</h3>
<hr>
<pre>
doc: 文件資料夾（純文字，utf-8）
qry: 查詢字串
</pre>

<hr>
<h3>詞法分析，工具函數定義（刪除非中文的所有文字與符號）</h3>

In [1]:
# 詞法分析，工具函數定義（刪除非中文的所有文字與符號）

import re

def remove_non_chinese(line):
    # 消除英文文數字
    rule = re.compile('[a-zA-Z0-9]')
    line = rule.sub(' ', line)
    # 消除特殊符號（含部分全形符號）
    rule = re.compile('[’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+')
    line = rule.sub(' ', line)
    # 消除不可見字碼
    rule = re.compile('[\001\002\003\004\005\006\007\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a]+')
    line = rule.sub(' ', line)
    # 消除所有全形符號
    rule = re.compile('[^\u4e00-\u9fa5]')
    line = rule.sub(' ', line)
    return line

def remove_redundant_space(line):
    line = re.sub(' +', ' ', line)
    return line


<hr>
<h3>詞法分析，載入 jieba 工具模組</h3>

In [2]:
# 詞法分析，載入 jieba 模組

import jieba

# 有必要的話載入常用辭典
jieba.set_dictionary('dict.txt.big')
# 有必要的話載入專屬字典
jieba.load_userdict('user.txt')


Building prefix dict from C:\Users\trchou\Dropbox\000 teaching\aisd10a\jupyter\unit_09_text\unit_09_text_tfidf_info_retrieval\dict.txt.big ...
Loading model from cache C:\Users\trchou\AppData\Local\Temp\jieba.uba41133c6d6ca814317cbdbde7c4be49.cache
Loading model cost 1.239 seconds.
Prefix dict has been built successfully.


<hr>
<h3>過濾無意義詞彙（stopwords，虛字與其它無意義詞彙）</h3>

In [3]:
# 過濾無意義詞彙（stopwords，虛字與其它無意義詞彙）

stopwords = [ '之', '乎', '者', '也', '的' ]

def remove_stopword(lst0):
    lst = []
    for x in lst0:
        if x not in stopwords:
            lst.append(x)
    return lst


<hr>
<h3>詞法分析，函數定義</h3>

In [4]:
# 詞法分析，函數定義

def lexical_analyzer(txt):
    txt = remove_non_chinese(txt)
    txt = remove_redundant_space(txt)
    res = jieba.cut(txt)
    lst = [ x for x in res ]
    lst = remove_stopword(lst)
    doc = ' '.join(lst)
    doc = re.sub(' +', ' ', doc)
    return doc
    

<hr>
<h3>整理 Corpus 資料格式</h3>

In [5]:
# 整理 Corpus 資料格式

import glob
import pickle

files = glob.glob('doc/*.txt')

corpus = dict()

for f in files:
    with open(f, 'r', encoding='utf-8') as fp:
        txt = fp.read()
    fp.close()
    corpus[f] = { 'name':f, 'txt':txt, 'doc':None }

for f in corpus:
    print(f)
    txt = corpus[f]['txt']
    doc = lexical_analyzer(txt)
    corpus[f]['doc'] = doc

with open('corpus_harry_potter.pkl', 'wb') as fp:
    pickle.dump(corpus, fp)
fp.close()


doc\哈利波特01神密的魔法石.txt
doc\哈利波特02消失的密室.txt
doc\哈利波特03阿茲卡班的逃犯.txt
doc\哈利波特04火盃的考驗(上).txt
doc\哈利波特05火盃的考驗(下).txt
doc\哈利波特06鳳凰會的密令.txt
doc\哈利波特07混血王子的背叛.txt
doc\哈利波特08死神的聖物.txt


<hr>
<h3>TF-IDF 文件檢索</h3>

In [6]:
# TF-IDF 文件檢索

from sklearn.feature_extraction.text import TfidfVectorizer

qry = '分類咒語'

vectorizer = TfidfVectorizer()

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

d = dict()
for f in corpus:
    text1 = lexical_analyzer(qry)
    text2 = corpus[f]['doc']
    c = cosine_sim(text1, text2)
    d[f] = c

# 字典排序（成為串列）
lst = sorted(d.items(), key=lambda x: x[1], reverse=True)

# 顯示排序結果
print('查詢：', qry)
for f, c in lst:
    t = '關連度：%12.10f，文章：%s' % (c, f)
    print(t)


查詢： 分類咒語
關連度：0.0164944566，文章：doc\哈利波特04火盃的考驗(上).txt
關連度：0.0131317313，文章：doc\哈利波特07混血王子的背叛.txt
關連度：0.0101660171，文章：doc\哈利波特06鳳凰會的密令.txt
關連度：0.0097156026，文章：doc\哈利波特02消失的密室.txt
關連度：0.0091124249，文章：doc\哈利波特05火盃的考驗(下).txt
關連度：0.0089375430，文章：doc\哈利波特01神密的魔法石.txt
關連度：0.0086871125，文章：doc\哈利波特08死神的聖物.txt
關連度：0.0043297963，文章：doc\哈利波特03阿茲卡班的逃犯.txt


<hr>
<h3 style="color:orange">『朱自清散文』語料的資料檢索平台建立（整合版）</h3>
<pre>
語料來源：
<a href="http://www.bwsk.net/mj/z/zhuziqing/index.html">http://www.bwsk.net/mj/z/zhuziqing/index.html</a>
</pre>

In [7]:
# 『朱自清散文』語料的資料檢索平台建立（整合版）

# 詞法分析，工具函數定義（刪除非中文的所有文字與符號）

import re

def remove_non_chinese(line):
    # 消除英文文數字
    rule = re.compile('[a-zA-Z0-9]')
    line = rule.sub(' ', line)
    # 消除特殊符號（含部分全形符號）
    rule = re.compile('[’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+')
    line = rule.sub(' ', line)
    # 消除不可見字碼
    rule = re.compile('[\001\002\003\004\005\006\007\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a]+')
    line = rule.sub(' ', line)
    # 消除所有全形符號
    rule = re.compile('[^\u4e00-\u9fa5]')
    line = rule.sub(' ', line)
    return line

def remove_redundant_space(line):
    line = re.sub(' +', ' ', line)
    return line

# 詞法分析，載入 jieba 模組

import jieba

# 有必要的話載入常用辭典
jieba.set_dictionary('dict.txt.big')
# 有必要的話載入專屬字典
jieba.load_userdict('user.txt')

# 無意義詞彙過濾（stopwords，虛字與其它無意義詞彙）

stopwords = [ '之', '乎', '者', '也', '的' ]

def remove_stopword(lst0):
    lst = []
    for x in lst0:
        if x not in stopwords:
            lst.append(x)
    return lst

# 詞法分析，函數定義

def lexical_analyzer(txt):
    txt = remove_non_chinese(txt)
    txt = remove_redundant_space(txt)
    res = jieba.cut(txt)
    lst = [ x for x in res ]
    lst = remove_stopword(lst)
    doc = ' '.join(lst)
    doc = re.sub(' +', ' ', doc)
    return doc

# 整理 Corpus 資料格式

import glob
import pickle

files = glob.glob('朱自清散文/*.txt')

corpus = dict()

for f in files:
    with open(f, 'r', encoding='utf-8') as fp:
        txt = fp.read()
    fp.close()
    corpus[f] = { 'name':f, 'txt':txt, 'doc':None }

# 儲存語料庫

for f in corpus:
    txt = corpus[f]['txt']
    doc = lexical_analyzer(txt)
    corpus[f]['doc'] = doc

with open('corpus_朱自清散文.pkl', 'wb') as fp:
    pickle.dump(corpus, fp)
fp.close()


Building prefix dict from C:\Users\trchou\Dropbox\000 teaching\aisd10a\jupyter\unit_09_text\unit_09_text_tfidf_info_retrieval\dict.txt.big ...
Loading model from cache C:\Users\trchou\AppData\Local\Temp\jieba.uba41133c6d6ca814317cbdbde7c4be49.cache
Loading model cost 1.043 seconds.
Prefix dict has been built successfully.


<hr>
<h3 style="color:orange">測試</h3>

In [8]:
# 測試

# 載入語料庫

with open('corpus_朱自清散文.pkl', 'rb') as fp:
    corpus = pickle.load(fp)
fp.close()

# TF-IDF 文件檢索

from sklearn.feature_extraction.text import TfidfVectorizer

qry = '月台+橘子'

vectorizer = TfidfVectorizer()

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

d = dict()
for f in corpus:
    text1 = lexical_analyzer(qry)
    text2 = corpus[f]['doc']
    c = cosine_sim(text1, text2)
    d[f] = c

# 字典排序（成為串列）
lst = sorted(d.items(), key=lambda x: x[1], reverse=True)

# 顯示排序結果
print('查詢：', qry)
for f, c in lst:
    t = '關連度：%12.10f，文章：%s' % (c, f)
    print(t)


查詢： 月台+橘子
關連度：0.1473410365，文章：朱自清散文\背影.txt
關連度：0.0000000000，文章：朱自清散文\《梅花》后記.txt
關連度：0.0000000000，文章：朱自清散文\一封信.txt
關連度：0.0000000000，文章：朱自清散文\儿女.txt
關連度：0.0000000000，文章：朱自清散文\匆匆.txt
關連度：0.0000000000，文章：朱自清散文\哀韋杰三君.txt
關連度：0.0000000000，文章：朱自清散文\女人.txt
關連度：0.0000000000，文章：朱自清散文\怀魏握青君.txt
關連度：0.0000000000，文章：朱自清散文\旅行雜記.txt
關連度：0.0000000000，文章：朱自清散文\槳聲燈影里的秦淮河.txt
關連度：0.0000000000，文章：朱自清散文\歌聲.txt
關連度：0.0000000000，文章：朱自清散文\海行雜記.txt
關連度：0.0000000000，文章：朱自清散文\溫州的蹤跡.txt
關連度：0.0000000000，文章：朱自清散文\白种人——上帝的驕子！.txt
關連度：0.0000000000，文章：朱自清散文\白采.txt
關連度：0.0000000000，文章：朱自清散文\航船中的文明.txt
關連度：0.0000000000，文章：朱自清散文\荷塘月色.txt
關連度：0.0000000000，文章：朱自清散文\說夢.txt
關連度：0.0000000000，文章：朱自清散文\阿河.txt
關連度：0.0000000000，文章：朱自清散文\飄零.txt
